In [1]:
from numpy import *
from pylab import *
from math import erf  # error function: erf(x)

In [2]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


D:\Program Files (x86)\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random', 'power', 'linalg', 'fft']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [3]:
# constant cell

r = 0.12  # risk-free interest
sigma = 0.09  # voltility
k = 50  # strike price

N = 300
s_start = 0
s_end = 150
s = linspace(s_start, s_end, N + 1)
ds = abs(s_end - s_start)/N  # ds, uniform grid

M = 8000
t_start = 0
t_end = 1  # expire time T
time = 0
t = linspace(t_start, t_end, M + 1)
dt = abs(t_end - t_start)/M  # dt, uniform grid

In [4]:
# list cell

# for all
V = [0 for i in range(N + 1)]  # exact solution V(s, T-t)
W = [0 for i in range(N + 1)]  # numerical solution W(s, T-t)
total_error = [0 for i in range(M + 1)] # total error


# for midpoint formula
s_first_partial = [0 for i in range(N + 1)]  # first partial of W
s_second_partial = [0 for i in range(N + 1)]  # second partial of W

# for spline
b = [0 for i in range(N + 1)]  # coefficient vector b
c = [0 for i in range(N + 1)]  # coefficient vector c
row_vector = [0 for i in range(N + 1)]  # for vector equation A
row_vector[0] = ds; row_vector[1] = 4*ds; row_vector[2] = ds;  # consider N > 2
v_vector = [0 for i in range(N + 1)]  # for vector equation z
spl_matrix = []  # matrix A

In [5]:
# function cell

# for all

def normal_dist(x): # standard normal distribution
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def CDF(t, idx):  # CDF of standard normal distribution
    arr = [0 for i in range(N + 1)]
    for i in range(1, N+1):
        if idx == 1:
            arr[i] = normal_dist((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
        else:
            arr[i] = normal_dist((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))    
    return arr

def exact_func(t):  # exact(analytic) function for fixed t
    return s*float_(CDF(t,1)) - k*exp(-r*t)*float_(CDF(t,2))

def abs_error(V,W):  # absolute error
    return abs(V-W)

# for midpoint formula
def aprx_func(V,W):  # approximation function of midpoint formula
    for i in range(1,N):
        s_first_partial[i] = (V[i+1] - V[i-1])/(2*ds) # midpoint formula for first order
        s_second_partial[i] = (V[i+1] - 2*V[i] + V[i-1])/ds**2 # midpoint formula for second order
    
    return 1/2 * sigma**2 * (s*s)*s_second_partial + r*s*s_first_partial - r*float_(W)

# for spline
def rotate(arr, n): # rotate the given arr to right n times 
    n %= len(arr)
    if not n:
        return arr
    
    left = arr[:-n]
    right = arr[-n:]
    
    return right + left

In [6]:
# set the matrix A

row_end = [0 for i in range(N + 1)]
row_end[0] = 1
spl_matrix.append(row_end)
spl_matrix.append(row_vector)

for i in range(2,N):
    row_vector = rotate(row_vector,1)
    spl_matrix.append(row_vector)
    
row_end = [0 for i in range(N + 1)]
row_end[N] = 1
spl_matrix.append(row_end)

In [7]:
### before check the 'numerical', run below cell first!

In [11]:
# set initial funciton
time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial graph
plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
axis((35, 65, 0, 21))
xlabel('stock price')
ylabel('call option price')

Text(0, 0.5, 'call option price')

In [10]:
# exact function

#initial function
time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        V[i] = s_start + i*ds - k
    else:
        V[i] = 0
        
# plot the graph        
plot(s,V, linewidth = 2.5, label = 'exact at τ='+str(round(time, 2)))
title('V at Remaining Time τ')
legend()
axis((35, 65, 0, 21))
xlabel('stock price')
ylabel('call option price')


while time < t_end:
    time = time + dt
    
    V = exact_func(time)
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,V, linewidth = 2.5, label = 'exact at τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,V, linewidth = 2.5, label = 'exact at τ='+str(round(time, 2)))
        legend()
                
    if isclose(time, 0.75):
        plot(s,V, linewidth = 2.5, label = 'exact at τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,V, linewidth = 2.5, label = 'exact at τ='+str(round(time, 2)))
        legend()
        
savefig("Project_exact.png")

In [9]:
# Euler method for midpoint formula: numerical

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    W = W + dt*aprx_func(W,W)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        title('W at Remaining Time τ: Midpoint Euler')
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        legend()
        
savefig("Project_numerical_midpoint.png")

In [12]:
# Euler method for spline
# it takes quite a while

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    W = W + dt*(r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        title('W at Remaining Time τ: Spline Euler')
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at τ='+str(round(time, 2)))
        legend()

savefig("Project_numerical_spline.png")

In [ ]:
### before check the 'error', run below cell first!

In [99]:
# set initial funciton
time = 0
e = 1

for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0

In [94]:
# Euler method for midpoint formula: error

clf()

# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
title('Error at Remaining Time τ: Midpoint Euler')
legend()
axis((30, 65, 0, 0.007))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    W = W + dt*aprx_func(W,W)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1.0):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
savefig("Project_error_midpoint.png")

In [96]:
# Euler method for spline: error
# it takes quite a while

clf()

# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
title('Error at Remaining Time τ: Spline Euler')
legend()
axis((30, 65, 0, 0.0025))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    W = W + dt*(r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 'τ='+str(round(time, 2)))
        legend()
        
savefig("Project_error_spline.png")

In [98]:
# midpoint global error

clf()
while time < t_end:
    time = time + dt
    
    # find the numerical solution
    W = W + dt*aprx_func(W,W)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    
    total_error[M-e] = sum(abs(V-W))
    e = e+1
    
total_error[M] = 0
plot(t,total_error,linestyle='--',linewidth = 1.8, label='Euler midpoint')
title('Global Error')
xlabel('time')
ylabel('global error')
legend()

In [100]:
# spline global error
# it takes quite a while

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    W = W + dt*(r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    
    total_error[M-e] = sum(abs(V-W))
    e = e+1
    
total_error[M] = 0
plot(t,total_error,linestyle='--',linewidth = 1.8, label='Euler spline')
legend()

savefig("Project_error_global.png")